In [2]:
%gui qt

from importlib import reload
 
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from pyqtgraph.flowchart import Flowchart
from pyqtgraph.Qt import QtGui, QtCore
from pyqtgraph.flowchart import library as fclib
from pyqtgraph.dockarea import Dock, DockArea

from plottr.data.datadict import togrid, DataDict, GridDataDict
from plottr.plot import mpl; reload(mpl)
from plottr.plot.mpl import ppcolormesh, PlotNode, MPLPlot, MPLPlotWidget
from plottr import test_data; reload(test_data)

<module 'plottr.test_data' from 'd:\\dropbox\\code\\labpydesktop\\plottr\\plottr\\test_data.py'>

# Notes

Developing a plotting window that has various analysis tools included (line cuts, and the like)

for mpl widgets, see:
* https://matplotlib.org/api/widgets_api.html
* https://matplotlib.org/_modules/matplotlib/widgets.html#PolygonSelector
* https://matplotlib.org/_modules/matplotlib/widgets.html

# Prototyping

In [20]:
from matplotlib.lines import Line2D
from matplotlib.widgets import _SelectorWidget, ToolHandles, Cursor, AxesWidget

class Line(_SelectorWidget):
    
    def __init__(self, ax, onselect, useblit=True):
        
        state_modifier_keys = dict(clear='c', move_vertex='control',
                                   move_all='not-applicable', 
                                   move='not-applicable',
                                   square='not-applicable',
                                   center='not-applicable')
        
        super().__init__(ax, onselect, useblit=useblit,
                         state_modifier_keys=state_modifier_keys)
        
        self._xs, self._ys = [], []
        
        lineprops = dict(color='k', linestyle='-', linewidth=2, alpha=0.5)
        lineprops['animated'] = self.useblit
        self.line = Line2D(self._xs, self._ys, **lineprops)
        self.ax.add_line(self.line)
        
        markerprops = dict(mec='k', mfc=lineprops.get('color', 'k'))
        self._handles = ToolHandles(self.ax, self._xs, self._ys,
                                    useblit=self.useblit,
                                    marker_props=markerprops)

        self._active_handle_idx = -1
#         self.vertex_select_radius = 15

        self.artists = [self.line, self._handles.artist]
        self.set_visible(True)
        
    def _press(self, event):
        pass
        
    def _release(self, event):
    
        if len(self._xs) == 2:
            for i in range(2):
                self._xs.pop(0)
                self._ys.pop(0)

        self._xs.insert(-1, event.x)
        self._ys.insert(-1, event.y)
            
        if len(self._xs) == 2:
            self.onselect(self.verts)
        
    def onmove(self, event):
        if not self.ignore(event):
            event = self._clean_event(event)
            self._onmove(event)
            return True
        return False
    
    def _onmove(self, event):
        pass
    
    
        
    @property
    def verts(self):
        """Get the line vertices.

        Returns
        -------
        list
            A list of the endpoints as ``(xdata, ydata)`` tuples.
        """
        return list(zip(self._xs, self._ys))


class ToolsPlot(MPLPlotWidget):
    
    def _plot2d(self, data, ax, xName, yName, dName):
        x = data[xName]['values']
        y = data[yName]['values']
        z = data[dName]['values']
        if isinstance(data, GridDataDict):
            z = z.T
            im = ppcolormesh(ax, x, y, z)
        else:
            im = ax.scatter(x, y, c=z)

        div = make_axes_locatable(ax)
        cax = div.append_axes("right", size="5%", pad=0.05)
        self.plot.fig.colorbar(im, cax=cax)

        ax.set_title(dName, size='small')
        ax.set_ylabel(data.label(yName))
        ax.set_xlabel(data.label(xName))
        cax.set_ylabel(data.label(dName))
    
    def setData(self, data):
        deps = data.dependents()
        axes = data.axes_list()
        
        if len(deps) != 1 or len(axes) != 2:
            raise ValueError("Incorrect data shape")
        
        self.ax = self.plot.clearFig(naxes=1)[0]
        
        self._plot2d(data, self.ax, axes[0], axes[1], deps[0])
        
        self.linetool = Line(self.ax, self.lineDrawn)
        
        
    def lineDrawn(self, verts):
        print(type(verts), verts)
        self.ax.figure.canvas.draw_idle()

In [21]:
nodelib = fclib.NodeLibrary()
nodelib.addNodeType(PlotNode, [('Plot')])

# library
fc = Flowchart(terminals={
    'dataIn': {'io': 'in'},
    'dataOut': {'io': 'out'}
})
fc.library = nodelib

plot = fc.createNode('Plot')
fc.connectTerminals(fc['dataIn'], plot['dataIn'])
fc.connectTerminals(plot['dataOut'], fc['dataOut'])


### Setting up the GUI

# plot
plotWidget = ToolsPlot()
plot.setPlotWidget(plotWidget)

# window
win = QtGui.QDialog()
area = DockArea()
layout = QtGui.QVBoxLayout()
layout.addWidget(area)
win.setLayout(layout)

# plot
plotDock = Dock('Plot')
plotDock.addWidget(plotWidget)
area.addDock(plotDock)

# show
win.show()

fc.setInput(dataIn=test_data.one_2d_set(51, 51).get_grid())

<class 'list'> [(297, 256), (192, 152)]
<class 'list'> [(373, 121), (133, 308)]
